# Imports and config

In [1]:
import os
import dspy
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API key loaded successfully")
else:
    print("API key not found in environment variables")


API key loaded successfully


# Configure the LM (Language Model)

In [2]:
# Configure the LM (Language Model) with structured output support
lm = dspy.LM("openai/gpt-4o-mini", model_type="chat")
dspy.settings.configure(lm=lm)

## Use DSPy built-in Module to Build a Sentiment Classifier

In [ ]:
class SentimentClassifier(dspy.Signature):
    """Classify the sentiment of a text."""

    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(
        desc="sentiment, the higher the more positive", ge=0, le=10
    )

# For my own notes: ge and le are pydantic constraints restricting the range of the sentiment output to be greater than or equal to: 0, less than or equal to: 10

In [ ]:
str_signature = dspy.make_signature("text -> sentiment")

# String based signature for the sentiment classifier (not recommended for production but good for testing)

## Create a Module to Interact with the LM

In [4]:
# Test the sentiment classifier
try:
    predict = dspy.Predict(SentimentClassifier)
    output = predict(text="I am feeling pretty happy about this!")
    print("Prediction successful!")
    print(f"Text: 'I am feeling pretty happy about this!'")
    print(f"Sentiment: {output.sentiment}")
except Exception as e:
    print(f"Error occurred: {e}")
    print("Please check your API key and internet connection.")

Prediction successful!
Text: 'I am feeling pretty happy about this!'
Sentiment: 8


In [5]:
# Test with different sentiment examples
test_texts = [
    "I am feeling pretty happy about this!",
    "This is terrible and I hate it.",
    "I feel neutral about this situation.",
    "I'm absolutely thrilled with the results!",
    "This makes me so angry and frustrated."
]

print("Testing sentiment classification with multiple examples:")

for text in test_texts:
    try:
        output = predict(text=text)
        print(f"Text: '{text}'")
        print(f"Sentiment: {output.sentiment}/10")
        print("." * 40)
    except Exception as e:
        print(f"Error processing '{text}': {e}")
        print("." * 40)


Testing sentiment classification with multiple examples:
Text: 'I am feeling pretty happy about this!'
Sentiment: 8/10
........................................
Text: 'This is terrible and I hate it.'
Sentiment: 0/10
........................................
Text: 'I feel neutral about this situation.'
Sentiment: 5/10
........................................
Text: 'I'm absolutely thrilled with the results!'
Sentiment: 10/10
........................................
Text: 'This makes me so angry and frustrated.'
Sentiment: 1/10
........................................


In [ ]:
# Alternative ways to view the output

print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is {output['sentiment']}")

The sentiment is: 1
The sentiment is 1


In [ ]:
# Changing the LM use the dspy.configure() function
dspy.configure(lm=dspy.LM("openai/gpt-4o"))
print(predict(text="I am feeling pretty happy!"))

Prediction(
    sentiment=8
)


In [8]:
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

Where is my prompt?
Check dspy.inspect_history(n=1) where n is how many entries you want to pull from the memory.

In [9]:
dspy.inspect_history(n=1)





[2025-10-24T18:30:30.110453]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with the corresponding output fields, starting with the field `[[ ## sentiment ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## sentiment ## ]]
8

[[ ## completed ## ]]







# Chain of Thought Built-in module.

In [11]:
cot = dspy.ChainOfThought(SentimentClassifier)

output = cot(text="I am feeling pretty happy!")
print(output)

Prediction(
    reasoning='The text expresses a positive emotion, specifically happiness. The use of the word "happy" indicates a strong positive sentiment. Therefore, the sentiment score is high.',
    sentiment=8
)


In [12]:
dspy.inspect_history(n=1)





[2025-10-24T18:31:15.361260]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str): 
2. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## sentiment ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
Th

## Using a different Adapter

In [13]:
dspy.configure(adapter=dspy.JSONAdapter())

In [14]:
print(cot(text="I am feeling pretty happy!"))
dspy.inspect_history(n=1)

Prediction(
    reasoning='The text expresses a positive emotion, specifically happiness, which indicates a strong positive sentiment.',
    sentiment=8
)




[2025-10-24T18:31:37.249582]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str): 
2. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "sentiment": "{sentiment}        # note: the value you produce must be a single int value"
}
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with a JSON object in the foll

## Building a Program with Custom Module

In [ ]:
class QuestionGenerator(dspy.Signature):
    """Genereate a yes or no question in order to guess the celebrity name"""
    past_questions: list[str] = dspy.InputField(desc="past questions ")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can")
    guess_made: bool = dspy.OutputField(desc="If the new_question is")

class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the celebrity")
    final_guessor_question: str = dspy.InputField(desc="the final guess")
    past_questions: list[str] = dspy.InputField(desc="past questions ")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OutputField(
        desc="reflection on the guessing process, including what was "
    )

def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")

class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_tries = 20

    def forward(self):
        celebrity_name = input("Please think of a celebrity name, once you are ready, type the name and press enter...")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers,
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't guess it right.")

        reflection = self.reflection(
            correct_celebrity_name=celebrity_name,
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)

In [17]:
celebrity_guess = CelebrityGuess()
celebrity_guess

question_generator.predict = Predict(StringSignature(past_questions, past_answers -> reasoning, new_question, guess_made
    instructions='Genereate a yes or no question in order to guess the celebrity name'
    past_questions = Field(annotation=list[str] required=True json_schema_extra={'desc': 'past questions ', '__dspy_field_type': 'input', 'prefix': 'Past Questions:'})
    past_answers = Field(annotation=list[bool] required=True json_schema_extra={'desc': 'past answers', '__dspy_field_type': 'input', 'prefix': 'Past Answers:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    new_question = Field(annotation=str required=True json_schema_extra={'desc': 'new question that can', '__dspy_field_type': 'output', 'prefix': 'New Question:'})
    guess_made = Field(annotation=bool required=True json_schema_extra={'desc': 'If the new_question is', '__d

## Save and Load using dspy

In [ ]:
celebrity_guess.save("dspy_program/celebrity.json", save_program=False)

In [26]:
celebrity_guess.load("dspy_program/celebrity.json")

In [22]:
celebrity_guess.save("dspy_program/celebrity/", save_program=True)

In [23]:
loaded = dspy.load("dspy_program/celebrity/")

In [24]:
loaded()

Yay! I got it right!
Throughout the guessing process, I focused on narrowing down the celebrity's identity by eliminating various fields of entertainment. The consistent answers indicating a connection to sports helped me hone in on the correct guess. I realized that asking about sports was a pivotal moment, as it confirmed the celebrity's primary domain. This reflection highlights the importance of strategic questioning and how each answer builds upon the last to lead to a more informed final guess.


In [27]:
dspy.inspect_history(n=1)





[2025-10-24T23:57:09.014270]

System message:

Your input fields are:
1. `correct_celebrity_name` (str): the celebrity
2. `final_guessor_question` (str): the final guess
3. `past_questions` (list[str]): past questions 
4. `past_answers` (list[bool]): past answers
Your output fields are:
1. `reasoning` (str): 
2. `reflection` (str): reflection on the guessing process, including what was
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## correct_celebrity_name ## ]]
{correct_celebrity_name}

[[ ## final_guessor_question ## ]]
{final_guessor_question}

[[ ## past_questions ## ]]
{past_questions}

[[ ## past_answers ## ]]
{past_answers}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "reflection": "{reflection}"
}
In adhering to this structure, your objective is: 
        Provide reflection on the guessing process


User message:

[[ ## correct_ce